In [61]:
# All necessary import 
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder,RobustScaler,LabelBinarizer,StandardScaler
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, Conv1D, MaxPool1D, Flatten, Dropout 
from keras.models import Sequential 
from keras.layers import Input
from keras.models import Model
import itertools
from itertools import cycle

In [62]:
df = pd.read_csv('../Data/preprocessed/123_processed.csv')
df

,durations,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0.000000,TCP,ajp,"PSH, ACK",110,20,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.008705,TCP,other,"PSH, ACK",110,20,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.062685,TCP,ajp,ACK,0,20,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.091396,TCP,ajp,"PSH, ACK",110,20,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.104046,TCP,other,"PSH, ACK",110,20,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
976,659.668482,TCP,other,"PSH, ACK",110,20,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
977,659.723065,TCP,ajp,ACK,0,20,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
978,659.723066,TCP,ajp,ACK,0,20,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
979,661.652531,TCP,http-alt,ACK,1,20,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
# Using Standard scaler for normalizing
std_scaler = StandardScaler()
def standardization(df,col):
  for i in col:
    arr = df[i]
    arr = np.array(arr)
    df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
  return df

numeric_col = df.select_dtypes(include='number').columns
data = standardization(df,numeric_col)



In [64]:

x = pd.get_dummies(df,columns=['protocol_type','service','flag'],prefix="",prefix_sep="")  
# let's get the names of our new columns
x_columns = x.columns
x_columns

Index(['durations', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
       'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
       'root_shell', 'su_attempted', 'num_root', 'num_file_creations',
       'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate', 'TCP', 'ajp', 'flexlm', 'gtalk', 'http-alt',
       'other', 'ACK', 'PSH, ACK'],
      dtype='object')

In [65]:
# let's read our txt file
with open('features.txt') as f:
    content = f.readlines()

In [66]:
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content]
# let's rename our columns using the txt file and create new columns if they don't exist
for i in content:
    if i not in x_columns:
        x[i] = 0 


C:\Users\youss\AppData\Local\Temp\ipykernel_23116\60187394.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  x[i] = 0
C:\Users\youss\AppData\Local\Temp\ipykernel_23116\60187394.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  x[i] = 0
C:\Users\youss\AppData\Local\Temp\ipykernel_23116\60187394.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-

In [67]:
x = x.reindex(columns=content)

In [68]:
df = pd.get_dummies(df,columns=['protocol_type','service','flag'],prefix="",prefix_sep="")  

x_test = x

In [69]:
x_test = x_test.astype(np.float32)

In [70]:
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
x_test.shape

(981, 122, 1)

In [71]:
from tensorflow.keras.models import load_model
#model_path = 'CNN_NSL-KDD.h5'
#model = load_model(model_path)
interpreter = tf.lite.Interpreter(model_path='CNN_NSL-KDD.tflite')
interpreter.allocate_tensors()



In [72]:
#prediction_array = model.predict(x_test)
# Assuming you have already loaded the TensorFlow Lite model and prepared the input tensor
interpreter.set_tensor(input_tensor_index, x_test)  # Set the input tensor
interpreter.invoke()  # Run inference
prediction_array = interpreter.get_tensor(output_tensor_index)  # Get the output tensor


31/31 [==============================] - 0s 2ms/step


In [73]:
# Mapping of classes
classes = np.array(['Dos', 'Probe', 'R2L', 'U2R', 'normal'])

# Get the index with the highest probability for each prediction
predicted_classes_indices = np.argmax(prediction_array, axis=1)

# Map the indices to the corresponding class labels
predicted_classes = classes[predicted_classes_indices]


In [74]:
x['Dos'] = prediction_array[:,0]
x['Probe'] = prediction_array[:,1]
x['R2L'] = prediction_array[:,2]
x['U2R'] = prediction_array[:,3]
x['normal'] = prediction_array[:,4]
x['predicted_classes'] = predicted_classes

C:\Users\youss\AppData\Local\Temp\ipykernel_23116\2810022988.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  x['Dos'] = prediction_array[:,0]
C:\Users\youss\AppData\Local\Temp\ipykernel_23116\2810022988.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  x['Probe'] = prediction_array[:,1]
C:\Users\youss\AppData\Local\Temp\ipykernel_23116\2810022988.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colum

In [75]:
x

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,S2,S3,SF,SH,Dos,Probe,R2L,U2R,normal,predicted_classes
0,0,0.750599,-0.264183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0.189812,0.196491,0.082014,0.042407,0.489275,normal
1,0,0.750599,-0.264183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0.185284,0.584568,0.057389,0.006179,0.166580,Probe
2,0,-1.097722,-0.264183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0.226214,0.228635,0.043280,0.013078,0.488793,normal
3,0,0.750599,-0.264183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0.189812,0.196491,0.082014,0.042407,0.489275,normal
4,0,0.750599,-0.264183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0.185284,0.584568,0.057389,0.006179,0.166580,Probe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
976,0,0.750599,-0.264183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0.185284,0.584568,0.057389,0.006179,0.166580,Probe
977,0,-1.097722,-0.264183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0.226214,0.228635,0.043280,0.013078,0.488793,normal
978,0,-1.097722,-0.264183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0.226214,0.228635,0.043280,0.013078,0.488793,normal
979,0,-1.080919,-0.264183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0.226661,0.227008,0.043911,0.013395,0.489024,normal
